In [1]:
import os
import kagglehub

# --- CONFIGURATION: PASTE YOUR KAGGLE CREDENTIALS HERE ---
# Open your kaggle.json file with a text editor to see these values
os.environ['KAGGLE_USERNAME'] = "meetparmar40"
os.environ['KAGGLE_KEY'] = "KGAT_c505d522d54ad468e6d6768c30208ffb"
# ---------------------------------------------------------

print("Authenticating with provided credentials...")

# Download the specific version of the dataset that contains the raw images
# This dataset (soumikrakshit/nyu-depth-v2) is the standard repackage of the 50k subset
try:
    path = kagglehub.dataset_download("soumikrakshit/nyu-depth-v2")
    print(f"\n✅ Success! Dataset downloaded to: {path}")
except Exception as e:
    print(f"\n❌ Error: {e}")
    print("Double check that your username and key are pasted correctly inside the quotes above.")

Authenticating with provided credentials...

❌ Error: POST failed with: {"errors":["New Datasets cannot be attached in non-interactive sessions. Found no versions attached for Dataset [soumikrakshit/nyu-depth-v2]."],"error":{"code":9},"wasSuccessful":false}
Double check that your username and key are pasted correctly inside the quotes above.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

NotImplementedError: Mounting drive is unsupported in this environment. Use PyDrive2 instead. See examples at https://colab.research.google.com/notebooks/io.ipynb#scrollTo=7taylj9wpsA2.

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T

class NYURawDataset(Dataset):
    def __init__(self, dataset_root, train=True):
        self.train = train

        # 1. Locate the CSV file automatically
        # We look for 'nyu2_train.csv' anywhere inside the downloaded folder
        csv_file = None
        for root, dirs, files in os.walk(dataset_root):
            if 'nyu2_train.csv' in files:
                csv_file = os.path.join(root, 'nyu2_train.csv')
                break

        if csv_file is None:
            raise FileNotFoundError("Could not find nyu2_train.csv. Check the dataset path.")

        print(f"Found Metadata CSV: {csv_file}")
        self.data_map = pd.read_csv(csv_file, header=None)

        # 2. AUTO-CALIBRATE PATHS (The Fix)
        # The CSV contains paths like: "/nyu_data/data/nyu2_train/basement_0001a_out/1.jpg"
        # We need to figure out how to map that to your actual local folder.

        # Get the first image path from the CSV
        sample_path_csv = self.data_map.iloc[0, 0] # e.g., /nyu_data/data/nyu2_train/...

        # Determine the file name (e.g., 1.jpg) and its immediate parent folder
        # This helps us search for the *actual* location of this specific file
        sample_filename = os.path.basename(sample_path_csv)
        sample_parent = os.path.basename(os.path.dirname(sample_path_csv))

        found_real_path = None
        # Walk the directory again to find where this specific sample image really lives
        for root, dirs, files in os.walk(dataset_root):
            if sample_filename in files and os.path.basename(root) == sample_parent:
                found_real_path = os.path.join(root, sample_filename)
                break

        if found_real_path is None:
            raise FileNotFoundError(f"Could not locate sample image {sample_filename} anywhere in dataset!")

        # 3. Calculate the 'Anchor'
        # Now we know where the file *is* (found_real_path) and what the CSV *says* (sample_path_csv).
        # We create a logic to merge them.

        print(f"Sample CSV Entry:  {sample_path_csv}")
        print(f"Actual File Found: {found_real_path}")

        # We will store the root that, when joined with the stripped CSV path, gives the real path.
        # Logic: We strip the common suffix from the real path.
        # CSV Path usually starts with: /nyu_data/data/nyu2_train/...
        # We clean the CSV path to be relative
        self.clean_csv_prefix = os.path.dirname(sample_path_csv).lstrip('/\\')

        # The dataset root for our purposes is the part of the real path *before* the CSV structure starts
        # This is a robust way to handle any nesting kagglehub or unzipping adds.
        if found_real_path.endswith(sample_filename):
             # Remove the filename to get directory
             real_dir = os.path.dirname(found_real_path)
             # If the CSV path structure matches the tail of the real path, we are good.
             if real_dir.endswith(self.clean_csv_prefix):
                 start_index = len(real_dir) - len(self.clean_csv_prefix)
                 self.img_root = real_dir[:start_index]
             else:
                 # Fallback: Just use the dataset_root and hope standard join works
                 self.img_root = dataset_root

        print(f"Calculated Image Root: {self.img_root}")

        self.normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    def __len__(self):
        return len(self.data_map)

    def __getitem__(self, idx):
        row = self.data_map.iloc[idx]
        img_rel = row[0].lstrip('/\\')
        depth_rel = row[1].lstrip('/\\')

        # Construct path using the calculated root
        img_path = os.path.join(self.img_root, img_rel)
        depth_path = os.path.join(self.img_root, depth_rel)

        try:
            image = Image.open(img_path).convert('RGB')
            depth = Image.open(depth_path)
        except FileNotFoundError:
            # If a specific file is missing, we print a warning and just return a random other one
            # to prevent crashing, but we DO NOT recurse endlessly.
            # print(f"Warning: Missing file {img_path}")
            return self.__getitem__(0) # Safety fallback to 0th element

        # Preprocessing
        transform_resize = T.Resize((224, 224)) # ResNet Standard Size
        image = transform_resize(image)
        depth = transform_resize(depth)

        img_tensor = T.ToTensor()(image)
        depth_tensor = T.ToTensor()(depth)

        if self.train:
            if torch.rand(1) < 0.5:
                img_tensor = T.functional.hflip(img_tensor)
                depth_tensor = T.functional.hflip(depth_tensor)
            if torch.rand(1) < 0.25:
                img_tensor = img_tensor[[1, 0, 2], :, :]

        img_tensor = self.normalize(img_tensor)
        depth_tensor = depth_tensor * 10.0 # Scale to 10m

        return img_tensor, depth_tensor

# --- RUN DATASET LOADING ---

# Ensure path variable exists from your previous kagglehub run
if 'path' not in globals():
    print("Please run the kagglehub download cell first to set 'path'")
else:
    try:
        # Initialize Dataset
        train_dataset = NYURawDataset(dataset_root=path, train=True)
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

        # Test valid batch
        imgs, depths = next(iter(train_loader))
        print("\n✅ SUCCESS! Data loaded correctly.")
        print(f"Batch Shape: {imgs.shape}")

    except Exception as e:
        print(f"\n❌ Setup Failed: {e}")

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split

# Ensure the dataset class from the previous step is loaded
if 'train_dataset' not in globals():
    raise NameError("Please run the 'NYURawDataset' class definition from the previous step first.")

# 1. Split the 50k dataset into Train (80%) and Validation (20%)
# The paper used ~50k for training, so we use the majority here.
total_size = len(train_dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

print(f"Total Samples: {total_size}")
print(f"Splitting into: {train_size} Train, {val_size} Validation")

# Set generator for reproducibility
generator = torch.Generator().manual_seed(42)
train_set, val_set = random_split(train_dataset, [train_size, val_size], generator=generator)

# 2. Create Loaders
# We use a batch size of 16 (or 8 if you run out of memory)
# Pin_memory=True helps speed up transfer to GPU
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

print("✅ DataLoaders created successfully.")

In [ ]:
# @title Block 2: Loss Function
# 1. Install the missing library first
!pip install -q kornia

import torch
import torch.nn as nn
import kornia

class CombinedLoss(nn.Module):
    def __init__(self, w1=0.1, w2=1.0, w3=0.5):
        super(CombinedLoss, self).__init__()
        self.w1 = w1 # Depth Pointwise Weight (Paper Eq 4)
        self.w2 = w2 # Gradient Weight (Paper Eq 4)
        self.w3 = w3 # SSIM Weight (Paper Eq 4)

        # Window size 11 is standard for SSIM
        self.ssim = kornia.losses.SSIMLoss(window_size=11, reduction='mean')

    def gradient_loss(self, pred, target):
        # Calculate gradients in X and Y directions (Paper Eq 2)
        pred_dx = torch.abs(pred[:, :, :, :-1] - pred[:, :, :, 1:])
        pred_dy = torch.abs(pred[:, :, :-1, :] - pred[:, :, 1:, :])
        target_dx = torch.abs(target[:, :, :, :-1] - target[:, :, :, 1:])
        target_dy = torch.abs(target[:, :, :-1, :] - target[:, :, 1:, :])
        return torch.mean(torch.abs(pred_dx - target_dx)) + torch.mean(torch.abs(pred_dy - target_dy))

    def forward(self, pred, target):
        # 1. Pointwise L1 Depth Loss (Paper Eq 1)
        l_depth = torch.mean(torch.abs(pred - target))

        # 2. Gradient Loss (Paper Eq 2)
        l_grad = self.gradient_loss(pred, target)

        # 3. SSIM Loss (Paper Eq 3)
        # Note: Kornia returns SSIM loss directly (which is usually 1 - SSIM)
        l_ssim = self.ssim(pred, target)

        # Total Weighted Sum (Paper Eq 4)
        return (self.w1 * l_depth) + (self.w2 * l_grad) + (self.w3 * l_ssim)

# Initialize Loss and Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = CombinedLoss().to(device)

print(f"✅ Loss function initialized on: {device}")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class UpSample(nn.Sequential):
    def __init__(self, skip_input, output_features):
        super(UpSample, self).__init__()
        self.convA = nn.Conv2d(skip_input, output_features, kernel_size=3, stride=1, padding=1)
        self.leakyreluA = nn.LeakyReLU(0.2)
        self.convB = nn.Conv2d(output_features, output_features, kernel_size=3, stride=1, padding=1)
        self.leakyreluB = nn.LeakyReLU(0.2)

    def forward(self, x, concat_with):
        # Upsample the input
        up_x = F.interpolate(x, size=[concat_with.size(2), concat_with.size(3)], mode='bilinear', align_corners=True)
        # Concatenate with the skip connection from the encoder
        return self.leakyreluB(self.convB(self.leakyreluA(self.convA(torch.cat([up_x, concat_with], dim=1)))))

class ResNetDepthModel(nn.Module):
    def __init__(self):
        super(ResNetDepthModel, self).__init__()

        # 1. ENCODER: Pre-trained ResNet-152
        # We load the weights to get "Transfer Learning" benefits
        original_model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)

        # Extract layers for Skip Connections
        self.encoder_layer0 = nn.Sequential(original_model.conv1, original_model.bn1, original_model.relu)
        self.encoder_layer1 = nn.Sequential(original_model.maxpool, original_model.layer1)
        self.encoder_layer2 = original_model.layer2
        self.encoder_layer3 = original_model.layer3
        self.encoder_layer4 = original_model.layer4

        # 2. DECODER: CNN with Upsampling
        # The channel numbers correspond to ResNet-152 feature map sizes
        self.up_block1 = UpSample(skip_input=2048 + 1024, output_features=1024)
        self.up_block2 = UpSample(skip_input=1024 + 512, output_features=512)
        self.up_block3 = UpSample(skip_input=512 + 256, output_features=256)
        self.up_block4 = UpSample(skip_input=256 + 64, output_features=128)

        # Final output layer (1 channel for Depth)
        self.final_conv = nn.Conv2d(128, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # --- Encoder Pass ---
        x0 = self.encoder_layer0(x) # Low level features
        x1 = self.encoder_layer1(x0)
        x2 = self.encoder_layer2(x1)
        x3 = self.encoder_layer3(x2)
        x4 = self.encoder_layer4(x3) # High level features (Bottleneck)

        # --- Decoder Pass (with Skips) ---
        d1 = self.up_block1(x4, x3)
        d2 = self.up_block2(d1, x2)
        d3 = self.up_block3(d2, x1)
        d4 = self.up_block4(d3, x0)

        # Final prediction
        return torch.sigmoid(self.final_conv(d4)) * 10.0 # Scale 0-1 output to 0-10m

# Initialize the correct model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetDepthModel().to(device)

print("✅ ResNet-152 Depth Model Initialized.")

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

# --- IMPORTANT: DEFINE MODEL BEFORE RUNNING THIS ---
if 'model' not in globals():
    # If you haven't defined the ResNet model yet, run this simple placeholder
    # In reality, you should use the ResNet-152 U-Net described in the paper
    print("⚠️ Warning: Initializing a basic U-Net placeholder. Replace with true ResNet model.")
    model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
                           in_channels=3, out_channels=1, init_features=32, pretrained=False)

model = model.to(device)
# --------------------------------------------------

# Learning rate from paper or standard default
optimizer = optim.Adam(model.parameters(), lr=0.0001)

def train_model(model, loader, criterion, optimizer, epochs=1):
    model.train()
    print(f"Starting training on {len(loader.dataset)} samples...")

    for epoch in range(epochs):
        running_loss = 0.0

        for i, (images, depths) in enumerate(loader):
            # Move data to GPU
            images, depths = images.to(device), depths.to(device)

            optimizer.zero_grad()

            # Forward Pass
            outputs = model(images)

            # Interpolate: ResNet encoders often output smaller feature maps (e.g. 1/2 size)
            # We resize output to match ground truth depth map size
            if outputs.shape[-2:] != depths.shape[-2:]:
                outputs = F.interpolate(outputs, size=depths.shape[-2:], mode='bilinear', align_corners=True)

            # Calculate Loss
            loss = criterion(outputs, depths)

            # Backward Pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Logging every 100 batches (approx every 1600 images)
            if i % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Step [{i}/{len(loader)}], Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(loader)
        print(f"Epoch {epoch+1} Complete. Average Loss: {epoch_loss:.4f}")

# Run Training
# Note: On 50k images, 1 epoch will take significantly longer than before.
train_model(model, train_loader, criterion, optimizer, epochs=1)

In [ ]:
# @title Upload & Visualize Your Own Image (Corrected)
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms # <--- This fixes your error
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image
import io

# Ensure device is defined (in case it wasn't earlier)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Upload Image
uploaded = files.upload()
filename = next(iter(uploaded))

# 2. Preprocess Image
input_image = Image.open(io.BytesIO(uploaded[filename])).convert('RGB')
original_size = input_image.size
input_image_resized = input_image.resize((640, 480)) # Resize to standard NYU size

# Convert to Tensor & Normalize
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = transform(input_image_resized).unsqueeze(0).to(device)

# 3. Predict
model.eval()
with torch.no_grad():
    pred = model(input_tensor)
    # Resize prediction back to original image size for quality
    pred = F.interpolate(pred, size=(original_size[1], original_size[0]), mode='bilinear', align_corners=True)

# 4. Visualize
depth_map = pred.squeeze().cpu().numpy()

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Your Image")
plt.imshow(input_image)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Predicted Depth Map")
# plt.imshow(depth_map, cmap='magma')
plt.imshow(depth_map, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# @title Visualize in Black & White (Like the Paper)
import matplotlib.pyplot as plt

# ... (Assuming you ran the previous prediction step) ...

depth_map = pred.squeeze().cpu().numpy()

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Your Image")
plt.imshow(input_image)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Predicted Depth Map (Grayscale)")

# CHANGE HERE: Use 'gray' instead of 'magma'
# 'gray_r' reverses it (White=Close, Black=Far) which often looks better
plt.imshow(depth_map, cmap='gray')

plt.axis('off')
plt.show()